In [1]:
import random
import math
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [2]:
class FuzzyController:
    def __init__(self, setpoint=5000):
        self.setpoint = setpoint  # Define setpoint as an attribute

        self.error = ctrl.Antecedent(np.arange(-100000, 100001, 1), 'error')
        self.pca = ctrl.Consequent(np.arange(-10, 11, 1), 'pca')

        # Define the membership functions for error
        self.error['negative_large'] = fuzz.trimf(self.error.universe, [-100000, -5000, -1500])
        self.error['negative_medium'] = fuzz.trimf(self.error.universe, [-5000, -1500, -500])
        self.error['negative_small'] = fuzz.trimf(self.error.universe, [-1500, -500, 0])
        self.error['zero'] = fuzz.trimf(self.error.universe, [-500, 0, 500])
        self.error['positive_small'] = fuzz.trimf(self.error.universe, [0, 500, 1500])
        self.error['positive_medium'] = fuzz.trimf(self.error.universe, [1000, 5000, 7000])
        self.error['positive_large'] = fuzz.trimf(self.error.universe, [5000, 7000, 100000])

        # Define the membership functions for prefetch count adjustment (pca)
        self.pca['higher_decrease'] = fuzz.trimf(self.pca.universe, [-10, -9, -8])
        self.pca['medium_decrease'] = fuzz.trimf(self.pca.universe, [-7, -6, -5])
        self.pca['small_decrease'] = fuzz.trimf(self.pca.universe, [-4, -3, -2])
        self.pca['zero'] = fuzz.trimf(self.pca.universe, [-1, 0, 1])
        self.pca['small_increase'] = fuzz.trimf(self.pca.universe, [2, 3, 4])
        self.pca['medium_increase'] = fuzz.trimf(self.pca.universe, [5, 6, 7])
        self.pca['higher_increase'] = fuzz.trimf(self.pca.universe, [8, 9, 10])

    def set_rules(self, rule_list):
        # Define fuzzy rules based on the given rule_list
        rules = []
        for rule in rule_list:
            antecedent, consequent = rule.split(' THEN ')
            error_level = antecedent.split('= ')[1].strip()
            pca_level = consequent.split('= ')[1].strip()
            rules.append(
                ctrl.Rule(self.error[error_level], self.pca[pca_level]))

        # Create the fuzzy control system
        self.control_system = ctrl.ControlSystem(rules)
        self.controller = ctrl.ControlSystemSimulation(self.control_system)

    def simulate(self, error_values):
        outputs = []
        for error in error_values:
            self.controller.input['error'] = error
            self.controller.compute()
            outputs.append(self.controller.output['pca'])
        return outputs

In [3]:
def crossover(parent1, parent2):
    # Ensure the parents have the same length
    if len(parent1) != len(parent2):
        raise ValueError("Parent rule sets must have the same length")

    # Choose a random crossover point
    crossover_point = random.randint(1, len(parent1) - 1)

    # Create offspring by combining rules from parents at the crossover point
    offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
    offspring2 = parent2[:crossover_point] + parent1[crossover_point:]

    return offspring1, offspring2


def roulette_wheel_selection(population, fitness_scores):
    total_fitness = sum(fitness_scores)
    selection_probs = [f / total_fitness for f in fitness_scores]

    # Generate cumulative probabilities
    cumulative_probs = []
    cumulative_sum = 0.0
    for prob in selection_probs:
        cumulative_sum += prob
        cumulative_probs.append(cumulative_sum)

    def select_one():
        r = random.random()
        for i, cum_prob in enumerate(cumulative_probs):
            if r <= cum_prob:
                return population[i]

    # Select two parents
    parent1 = select_one()
    parent2 = select_one()

    return parent1, parent2


def mutate(parent, mutation_rate):
    pca_labels = ['higher_decrease', 'medium_decrease', 'small_decrease',
                  'zero', 'small_increase', 'medium_increase', 'higher_increase']
    mutated = parent.copy()
    for i in range(len(mutated)):
        if random.random() < mutation_rate:
            antecedent, consequent = mutated[i].split(' THEN ')
            error_level = antecedent.split('= ')[1].strip()
            pca_level = consequent.split('= ')[1].strip()
            error_index = ['negative_large', 'negative_medium', 'negative_small', 'zero',
                           'positive_small', 'positive_medium', 'positive_large'].index(error_level)
            pca_index = pca_labels.index(pca_level)
            error_index = (error_index + random.randint(-1, 1)) % 7
            pca_index = (pca_index + random.randint(-1, 1)) % 7
            mutated[i] = f"IF ERROR = {error_level} THEN PCA = {pca_labels[pca_index]}"
    return mutated


def calculate_fitness_from_df(rule_set, df, df_reference, lower_bound, upper_bound, setpoint=4500):
    controller = FuzzyController(setpoint)
    controller.set_rules(rule_set)
    df['controller_output'] = controller.simulate(df['error'])
    # Calculate the deviation from the prefetch_count
    df['new_prefetch_count'] = round(
        df['controller_output'] + df['prefetch_count'])
    # If new_prefetch_count is less than the lower bound, set it to the lower bound
    df['new_prefetch_count'] = df['new_prefetch_count'].apply(
        lambda x: lower_bound if x < lower_bound else x)
    # If new_prefetch_count is greater than the upper bound, set it to the upper bound
    df['new_prefetch_count'] = df['new_prefetch_count'].apply(
        lambda x: upper_bound if x > upper_bound else x)
    # Check if the new prefetch count is within the acceptable range using the reference dataframe
    df['new_arrival_rate'] = df['new_prefetch_count'].map(
        df_reference['arrival_rate'])
    # calculate nmse for the deviation
    max_arrival_rate = df_reference['arrival_rate'].max()
    min_arrival_rate = df_reference['arrival_rate'].min()
    df['deviation'] = df['new_arrival_rate'] - setpoint
    df['mse'] = df['deviation'] ** 2
    mse = df['mse'].mean()
    rmse = math.sqrt(mse)
    nrmse = rmse / (max_arrival_rate - min_arrival_rate)

    return nrmse

In [4]:
initial_population = [
    [
        'IF ERROR = negative_large THEN PCA = higher_decrease', 
        'IF ERROR = negative_medium THEN PCA = small_decrease',
        'IF ERROR = negative_small THEN PCA = small_decrease', 
        'IF ERROR = zero THEN PCA = zero', 
        'IF ERROR = positive_small THEN PCA = small_increase', 
        'IF ERROR = positive_medium THEN PCA = small_increase', 
        'IF ERROR = positive_large THEN PCA = higher_increase'
    ],
    [
        'IF ERROR = negative_large THEN PCA = higher_decrease', 
        'IF ERROR = negative_medium THEN PCA = medium_decrease',
        'IF ERROR = negative_small THEN PCA = small_decrease', 
        'IF ERROR = zero THEN PCA = zero', 
        'IF ERROR = positive_small THEN PCA = small_increase', 
        'IF ERROR = positive_medium THEN PCA = medium_increase', 
        'IF ERROR = positive_large THEN PCA = higher_increase'
    ],
    [
        'IF ERROR = negative_large THEN PCA = higher_decrease', 
        'IF ERROR = negative_medium THEN PCA = medium_decrease', 
        'IF ERROR = negative_small THEN PCA = small_decrease', 
        'IF ERROR = zero THEN PCA = zero', 
        'IF ERROR = positive_small THEN PCA = small_increase', 
        'IF ERROR = positive_medium THEN PCA = higher_increase', 
        'IF ERROR = positive_large THEN PCA = higher_increase'
    ],
    [
        'IF ERROR = negative_large THEN PCA = medium_decrease', 
        'IF ERROR = negative_medium THEN PCA = higher_decrease', 
        'IF ERROR = negative_small THEN PCA = medium_decrease', 
        'IF ERROR = zero THEN PCA = zero', 
        'IF ERROR = positive_small THEN PCA = small_increase', 
        'IF ERROR = positive_medium THEN PCA = medium_increase', 
        'IF ERROR = positive_large THEN PCA = higher_increase'
    ],
    [
        'IF ERROR = negative_large THEN PCA = higher_decrease', 
        'IF ERROR = negative_medium THEN PCA = medium_decrease', 
        'IF ERROR = negative_small THEN PCA = small_decrease', 
        'IF ERROR = zero THEN PCA = zero', 
        'IF ERROR = positive_small THEN PCA = higher_increase', 
        'IF ERROR = positive_medium THEN PCA = small_increase', 
        'IF ERROR = positive_large THEN PCA = medium_increase'
    ],
    [
        'IF ERROR = negative_large THEN PCA = higher_decrease', 
        'IF ERROR = negative_medium THEN PCA = small_decrease', 
        'IF ERROR = negative_small THEN PCA = medium_decrease', 
        'IF ERROR = zero THEN PCA = zero', 
        'IF ERROR = positive_small THEN PCA = medium_increase', 
        'IF ERROR = positive_medium THEN PCA = higher_increase', 
        'IF ERROR = positive_large THEN PCA = small_increase'
    ],
    [
        'IF ERROR = negative_large THEN PCA = higher_decrease', 
        'IF ERROR = negative_medium THEN PCA = medium_decrease', 
        'IF ERROR = negative_small THEN PCA = higher_decrease', 
        'IF ERROR = zero THEN PCA = zero', 
        'IF ERROR = positive_small THEN PCA = small_increase', 
        'IF ERROR = positive_medium THEN PCA = medium_increase', 
        'IF ERROR = positive_large THEN PCA = higher_increase'
    ],
    [
        'IF ERROR = negative_large THEN PCA = higher_decrease', 
        'IF ERROR = negative_medium THEN PCA = medium_decrease', 
        'IF ERROR = negative_small THEN PCA = small_decrease', 
        'IF ERROR = zero THEN PCA = zero', 
        'IF ERROR = positive_small THEN PCA = higher_increase', 
        'IF ERROR = positive_medium THEN PCA = higher_increase', 
        'IF ERROR = positive_large THEN PCA = medium_increase'
    ],
    [
        'IF ERROR = negative_large THEN PCA = medium_decrease', 
        'IF ERROR = negative_medium THEN PCA = small_decrease', 
        'IF ERROR = negative_small THEN PCA = higher_decrease', 
        'IF ERROR = zero THEN PCA = zero', 
        'IF ERROR = positive_small THEN PCA = medium_increase', 
        'IF ERROR = positive_medium THEN PCA = small_increase', 
        'IF ERROR = positive_large THEN PCA = higher_increase'
    ],
    [
        'IF ERROR = negative_large THEN PCA = higher_decrease', 
        'IF ERROR = negative_medium THEN PCA = higher_increase', 
        'IF ERROR = negative_small THEN PCA = small_decrease', 
        'IF ERROR = zero THEN PCA = zero', 
        'IF ERROR = positive_small THEN PCA = medium_increase', 
        'IF ERROR = positive_medium THEN PCA = small_increase', 
        'IF ERROR = positive_large THEN PCA = higher_increase'
    ]
]

In [5]:
df = pd.read_csv('results.csv')
df_reference = df.groupby('prefetch_count').mean().drop(columns=['sample_number', 'setpoint'])
df['error'] = df['arrival_rate'] - 4500
# create a new df grouped by the prefetch_count
lower_bound = df['prefetch_count'].min()
upper_bound = df['prefetch_count'].max()

In [6]:
# min and max values for the prefetch_count
df_reference

,arrival_rate
prefetch_count,
1,2805.011698
2,4679.087817
3,6251.915124
4,6695.801467
5,6837.105641
6,6840.902720


In [7]:
best_fitness = math.inf

# Run the genetic algorithm for 100 generations or until the best fitness is <100

for generation in range(100):
    # Calculate the fitness of each individual in the population
    fitness_scores = []
    for rule_set in initial_population:
        fitness = calculate_fitness_from_df(rule_set, df, df_reference,lower_bound, upper_bound)
        fitness_scores.append(fitness)

    # Find the best individual in the population
    best_index = np.argmin(fitness_scores)
    best_rule_set = initial_population[best_index]
    best_fitness = fitness_scores[best_index]

    # Check if the best fitness is less than 0.2
    if best_fitness < 0.2:
        break

    # Select parents for crossover
    parent1, parent2 = roulette_wheel_selection(initial_population, fitness_scores)

    # Perform crossover to generate offspring
    offspring1, offspring2 = crossover(parent1, parent2)

    # Mutate the offspring
    offspring1 = mutate(offspring1, 0.3)
    offspring2 = mutate(offspring2, 0.3)

    # Replace the worst individual in the population with the offspring
    worst_index = np.argmax(fitness_scores)
    initial_population[worst_index] = offspring1
    worst_index = np.argmax(fitness_scores)
    initial_population[worst_index] = offspring2

    print(f"Generation {generation}: Best Fitness = {best_fitness}")

Generation 0: Best Fitness = 0.5233740306257373
Generation 1: Best Fitness = 0.5233740306257373
Generation 2: Best Fitness = 0.5233740306257373
Generation 3: Best Fitness = 0.5233740306257373
Generation 4: Best Fitness = 0.5233740306257373
Generation 5: Best Fitness = 0.5233740306257373
Generation 6: Best Fitness = 0.5233740306257373
Generation 7: Best Fitness = 0.5233740306257373
Generation 8: Best Fitness = 0.5233740306257373
Generation 9: Best Fitness = 0.5233740306257373
Generation 10: Best Fitness = 0.5233740306257373
Generation 11: Best Fitness = 0.5233740306257373
Generation 12: Best Fitness = 0.41831203877331724
Generation 13: Best Fitness = 0.41831203877331724
Generation 14: Best Fitness = 0.41831203877331724
Generation 15: Best Fitness = 0.41831203877331724
Generation 16: Best Fitness = 0.41831203877331724
Generation 17: Best Fitness = 0.41831203877331724
Generation 18: Best Fitness = 0.38035594732041555
Generation 19: Best Fitness = 0.38035594732041555
Generation 20: Best Fi

- An NMSE close to 0 indicates a very accurate controller.
- An NMSE around 1 indicates a controller performing on par with desired mean.
- An NMSE greater than 1 indicates a model that is less accurate than adjusting PC of the observed values.

In [8]:
best_fitness

0.33739487104551524

In [9]:
best_rule_set

['IF ERROR = negative_large THEN PCA = higher_increase',
 'IF ERROR = negative_medium THEN PCA = medium_decrease',
 'IF ERROR = negative_small THEN PCA = zero',
 'IF ERROR = zero THEN PCA = zero',
 'IF ERROR = positive_small THEN PCA = zero',
 'IF ERROR = positive_medium THEN PCA = small_decrease',
 'IF ERROR = positive_large THEN PCA = higher_decrease']